# (Individualize) Plotting script

In [ ]:
import os
os.chdir("/home/s371513/ernie")

import models

import pickle as pkl
import pandas as pd

from tqdm import tqdm
from plotting.wrapper import wrap_plot as plot

chunk_size = 10000



In [ ]:
with open(os.path.join("custom_topics.pkl"), "rb") as t:
    custom_topics = pkl.load(t)

    dfs = []

    documents = os.listdir("data/out/noto")

    file_chunks = [documents[i:i + chunk_size] for i in range(0, len(documents), chunk_size)]

    c = 1
    for file_chunk in file_chunks:
        loop = tqdm(file_chunk)
        loop.set_description_str("Batch {} of {}".format(c, len(file_chunks)))

        docs = []

        corrupted_files = 0

        for doc_file in loop:
            with open(os.path.join("data/out/noto", doc_file), "rb") as d:
                try:
                    document = pkl.load(d)

                    if document.topic_information is not None:
                        docs.append(document)

                except EOFError as _:
                    corrupted_files += 1
                    loop.set_postfix_str("Corrupted files: {}".format(corrupted_files))

                d.close()

        chunk_df = pd.DataFrame([vars(doc) for doc in docs])
        dfs.append(chunk_df)

        c += 1

    df = pd.concat(dfs)

Batch 7 of 18:  22%|██▏       | 2216/10000 [00:03<00:41, 188.64it/s] 

In [ ]:
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d%H%M%S")
df["date"] = df["date"].dt.round('1h')